<a href="https://colab.research.google.com/github/12jeong/Kaggle-beginner/blob/master/Santander-Product-Recommendation-Competition/D02_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.python.client import device_lib
import platform
from google.colab import drive
drive.mount('/content/drive/')  

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# For Jupyter Notebook
%matplotlib inline
import seaborn as sns
os.getcwd()
os.chdir('/content/drive/My Drive/Colab Notebooks/Kaggle-beginner')

In [0]:
import xgboost as xgb
import pickle 

In [3]:
trn = pd.read_csv('train_ver2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
tst = pd.read_csv('test_ver2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1. 데이터전처리

In [0]:
# 상품 변수
prods = trn.columns[24:].tolist()

# 상품 변수 결측값을 0으로 대체 (구매 하지 않았다고 가정)
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 노이즈제거 - 구매한 상품이 하나도 없는 고객의 경우
no_product = trn[prods].sum(axis=1) == 0 # sum(axis=1) : 행단위 계산
trn = trn[-no_product]

# 훈련데이터, 테스트 데이터 통합 (왜?)
# 테스트 데이터에 없는 훈련데이터 상품은 모두 0값으로 대체
for col in trn.columns[24:] :
    tst[col] = 0 
df = pd.concat([trn,tst], axis=0)

In [0]:
# 학습에 사용할 변수 목록
# variable_note.md 참고
features = []

# 범주형 변수
categorical_cols = ['ind_empleado','pais_residencia','sexo','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall','tipodom','nomprov','segmento']
features += categorical_cols
for col in categorical_cols :
    df[col],_ = df[col].factorize(na_sentinel=-99) # _ 는 unique 항 값을 무시하고 싶어서 사용하는 듯

In [7]:
# 참고
label, unique = pd.factorize(['a','b','c','d','a','d',np.NaN],na_sentinel=-99)
print("\n\nNumeric Representation : \n", label) 
print("Unique Values : \n", unique) 



Numeric Representation : 
 [  0   1   2   3   0   3 -99]
Unique Values : 
 ['a' 'b' 'c' 'd']


In [0]:
# 수치형 변수
numerical_cols = ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

# 결측치 대체
for col in numerical_cols:
    df[col].fillna(-99,inplace=True)

In [9]:
# 수치형으로 바꿀 수 없는 특이값 찾기
for col in numerical_cols:
    col_unique = df[col].unique()
    for i in range(len(col_unique)):
        try: 
            x = float(col_unique[i])
        except ValueError as e:
            print(col,col_unique[i],e)

age  NA could not convert string to float: ' NA'
antiguedad      NA could not convert string to float: '     NA'
renta          NA could not convert string to float: '         NA'
indrel_1mes P could not convert string to float: 'P'


In [0]:
# 특이값대체
# 메모리 효율을 위해 정수형으로 변환

df['age'].replace(' NA',-99,inplace=True) 
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA',-99,inplace=True)
df['antiguedad'].replace('-999999',-99,inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA',-99,inplace=True)
df['renta'].fillna(-99,inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P',5,inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

features = features + numerical_cols

### 2. 피처 엔지니어링 (파생 변수 생성)

- fecha_alta_month(첫 계약을 맺은 날짜), ult_fec_cli_1t(마지막으로 1등급이었던 날짜)에 대해 연,월 추출 변수 생성

In [0]:
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)  # object에서 float는 nan을 거르는 용도인듯?
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int8)                                                          

In [0]:
df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)  
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int8)                                                          

In [13]:
# 참고
x=float(26)
y='abc'
z=np.nan
print(x.__class__, y.__class__, z.__class__)

<class 'float'> <class 'str'> <class 'float'>


In [0]:
# 그 외 변수 결측값 대체
df.fillna(-99,inplace=True)

In [25]:
# 참고
a = " 2015-01-28 "
print('str:',a,'len:',len(a))
print('str:',a.strip(),'len:',len(a.strip()))

str:  2015-01-28  len: 12
str: 2015-01-28 len: 10


In [0]:
# lag-1(month) 상품 보유 생성
def date_to_int(str_date):
    Y,M,D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y)-2015)*12+int(M)
    return int_date

df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

df_lag = df.copy()
df_lag.columns = [col+'_prev' if col not in ['ncodpers','int_date'] else col for col in df.columns]
df_lag['int_date'] +=1

del trn,tst
df_trn = df.merge(df_lag, on=['ncodpers','int_date'],how="left")

In [0]:
del df, df_lag

In [0]:

for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)

features += [feature +'_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

In [0]:
print(features)

In [0]:
df_trn.to_csv('df_trn.csv',index=False)

In [0]:
import pickle 

In [0]:
filename = "feature_list.pickle"
# save
with open(filename, 'wb') as f:
    pickle.dump(features, f)
    pickle.dump(prods,f)
    
# load
#with open('data.pickle', 'rb') as f:
#    data = pickle.load(f)